# Amazon Bedrock AgentCore Policy - NL2Cedar 데모

## 개요

Amazon Bedrock AgentCore Policy 핸즈온 데모에 오신 것을 환영합니다! 이 노트북은 자연어로부터 Cedar policy를 생성하는 전체 워크플로우를 안내합니다. 이 노트북은 생성할 수 있는 다양한 유형의 policy를 시연하고 다양한 policy 구조를 이해하는 방법을 보여줍니다.

### Amazon Bedrock AgentCore Policy의 Natural Language Authoring이란?

NL2Cedar - Natural Language Authoring of Cedar Policies는 자연어로 권한 요구사항을 작성하면 자동으로 Cedar 구문으로 변환하고 생성된 policy가 요구사항과 일치하는지 검증합니다.

---

## 사전 요구사항

시작하기 전에 다음을 확인하세요:

- ✅ 적절한 자격 증명으로 구성된 AWS CLI
- ✅ boto3가 설치된 Python 3.10+
- ✅ `bedrock_agentcore_starter_toolkit` 패키지 설치
- ✅ AWS Lambda 액세스 권한 (대상 함수용)

01-Getting-Started/AgentCore-Policy-Demo.ipynb는 3개의 Lambda 대상이 있는 Insurance Underwriting 사용 사례를 위한 Gateway를 설정합니다. 동일한 gateway 설정을 사용할 것입니다.

시작해봅시다! 🚀

---

# Step 0: 환경 설정

먼저 환경을 확인하고 필요한 라이브러리를 가져옵니다.

In [ ]:
%pip install -r requirements.txt

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import subprocess
import boto3
import json
import logging
import time

# scripts 디렉토리를 Python path에 추가
scripts_dir = Path.cwd() / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

# AWS region 확인
session = boto3.Session()
region = session.region_name or "us-east-1"

# AWS 자격 증명 확인
try:
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ AWS Credentials Verified")
    print(f"   Account: {identity['Account']}")
    print(f"   User/Role: {identity['Arn']}")
except Exception as e:
    print(f"❌ AWS Credentials Error: {e}")
    print("   Please configure AWS CLI with: aws configure")

### 08-AgentCore-policy/01-Getting-Started/AgentCore-Policy-Demo.ipynb의 Gateway 설정이 존재하는지 확인합니다. 존재하지 않으면 다음 단계에서 3개의 Lambda 대상이 있는 insurance underwriting용 gateway를 설정합니다

In [ ]:
# Check if Gateway for Insurance Underwriting exists

# Getting-Started 디렉토리 경로 설정
current_dir = Path.cwd()
getting_started_dir = current_dir.parent / "01-Getting-Started"
config_file = getting_started_dir / "config.json"
scripts_dir = getting_started_dir / "scripts"

print("🔍 Checking for configuration...")
print(f"Looking for: {config_file.relative_to(current_dir.parent.parent)}")

if config_file.exists():
    print("✅ Configuration file found!")

    # 필수 필드 검증
    import json

    try:
        with open(config_file, "r") as f:
            config = json.load(f)

        required_fields = ["lambdas", "gateway", "region"]
        missing_fields = [field for field in required_fields if field not in config]

        if missing_fields:
            print(f"⚠️  Warning: Config file is missing fields: {missing_fields}")
            print("   You may need to re-run the setup scripts.")
        else:
            print("✅ Configuration is complete!")

    except json.JSONDecodeError:
        print("⚠️  Warning: Config file exists but is not valid JSON")

else:
    print("❌ Configuration file not found!")
    print("\n" + "=" * 70)
    print("Setting up infrastructure...")
    print("=" * 70)

    # 스크립트 실행을 위해 Getting-Started 디렉토리로 이동
    os.chdir(getting_started_dir)

    try:
        # Step 1: Lambda 함수 배포
        print("\n📦 Step 1: Deploying Lambda functions...")
        print("-" * 70)
        deploy_lambda_script = scripts_dir / "lambda-target-setup" / "deploy_lambdas.py"
        result = subprocess.run(
            [sys.executable, str(deploy_lambda_script)], capture_output=True, text=True
        )
        print(result.stdout)

        # Step 2: Gateway 설정
        print("\n🌐 Step 2: Setting up AgentCore Gateway...")
        print("-" * 70)
        setup_gateway_script = scripts_dir / "setup_gateway.py"
        result = subprocess.run(
            [sys.executable, str(setup_gateway_script)], capture_output=True, text=True
        )
        print(result.stdout)

        print("\n" + "=" * 70)
        print("✅ Infrastructure setup complete!")
        print("=" * 70)

        # config 파일 생성 확인
        if config_file.exists():
            print(f"✅ Configuration file created: {config_file}")
        else:
            print("⚠️  Warning: Setup completed but config.json was not created")

    except Exception as e:
        print(f"\n❌ Setup failed: {e}")
        print("\nPlease run the setup scripts manually:")
        print(f"1. cd {getting_started_dir}")
        print("2. python scripts/lambda-target-setup/deploy_lambdas.py")
        print("3. python scripts/setup_gateway.py")
    finally:
        # 원래 디렉토리로 복귀
        os.chdir(current_dir)

print("\n" + "=" * 70)

---

# Step 1: Policy Engine 생성

이제 Cedar policy를 보관할 Policy Engine을 생성합니다

Policy Engine은 policy의 모음입니다. 인바운드 트래픽에 대한 policy의 실시간 적용을 위해 gateway와 연결할 수 있습니다.
다음 단계에서 이 policy engine에 policy를 생성할 것입니다

### Policy Engine 생성

먼저 Cedar policy를 보관할 Policy Engine을 생성합니다:

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

policy_client = PolicyClient(region_name=region)
policy_client.logger.setLevel(logging.INFO)

# Policy Engine 생성
print("🔧 Creating Policy Engine...")
engine = policy_client.create_or_get_policy_engine(
    name="InsurancePolicyEngine_NL2Cedar",
    description="Policy engine for insurance underwriting governance",
)
print(f"✓ Policy Engine: {engine['policyEngineId']}\n")

# Policy Engine 정보를 config 파일에 저장
with open(config_file, "r") as f:
    config = json.load(f)

# 기존 데이터를 유지하면서 policy engine 정보 추가
config["policy_engine_id"] = engine["policyEngineId"]
config["policy_engine_arn"] = engine["policyEngineArn"]

# 업데이트된 config 저장
with open(config_file, "w") as f:
    json.dump(config, f, indent=2)

## Step 2: 자연어로부터 policy 생성

이제 NL2Cedar 기능을 사용하여 자연어로부터 Cedar policy를 생성합니다. NL2Cedar를 사용한 policy 생성은 두 단계로 이루어집니다: 먼저 자연어로부터 Cedar policy를 생성하고, 그 다음 Policy engine에 policy를 생성합니다.

> **💡 팁**: Gateway의 대상 스키마가 NL2Cedar 기능에 제공되어 foundation model이 대상의 이름과 파라미터를 이해하는 데 도움을 줍니다.

Gateway에는 3개의 Lambda 대상이 있습니다:
1. Application Tool: 간소화된 Application 생성 (데모 목적으로 모의 구현)
 신청자 지역 및 보장 금액으로 보험 신청서를 생성합니다
 파라미터:
 - applicant_region: 고객의 지리적 지역
 - coverage_amount: 요청된 보험 보장 금액

2. Risk Model Tool: 간소화된 Risk Model 액세스 (데모 목적으로 모의 구현)
 위험 점수 model을 호출하고 평가를 반환합니다
 파라미터:
 - API_classification: API 분류 (public, internal, restricted)
 - data_governance_approval: 데이터 거버넌스가 model 사용을 승인했는지 여부

3. Approval Tool: - 보험 승인 프로세스 (데모 목적으로 모의 구현)
 인수 결정 및 청구 금액을 승인합니다
 파라미터:
 - claim_amount: 보험 청구/보장 금액
 - risk_level: 위험 수준 평가 (low, medium, high, critical)


 우리의 자연어 문장은 이러한 대상 중 하나를 참조할 수 있으며 이러한 파라미터를 기반으로 agent-tool 액세스에 제약을 도입할 수 있습니다.

In [ ]:
# Cedar policy 생성
print("\n📝 Generating Cedar Policy from Natural language...")
print("\n📝 Simple natural language statement")

nl_input = "Allow all users to invoke the application tool when the coverage amount is under 1 million and the application region is US or CAN"

# 자연어로부터 Cedar policy 생성
result = policy_client.generate_policy(
    policy_engine_id=config["policy_engine_id"],
    name=f"nl_policy_{int(time.time())}",
    resource={"arn": config["gateway"]["gateway_arn"]},
    content={"rawText": nl_input},
    fetch_assets=True,  # Gateway의 대상 스키마를 가져와 foundation model에 제공
)

In [ ]:
# 생성된 Cedar policy 출력
if result.get("status") == "GENERATED" and result.get("generatedPolicies"):
    generated_policy = result["generatedPolicies"][0]
    cedar_statement = (
        generated_policy.get("definition", {}).get("cedar", {}).get("statement", "N/A")
    )

    print("Generated Cedar Policy:")
    print("=" * 60)
    print(cedar_statement)
    print("=" * 60)

## Step 2: 생성된 Cedar policy로부터 policy 생성

In [ ]:
# 생성된 Cedar policy로부터 policy 생성
if result.get("status") == "GENERATED" and result.get("generatedPolicies"):
    generated_policy = result["generatedPolicies"][0]
    application_creation_policy = policy_client.create_policy(
        policy_engine_id=config["policy_engine_id"],
        name="application_creation_policy",
        description="Allow application creation when coverage is under $1M and region is US or CA",
        definition=generated_policy.get("definition", {}),
    )

---

# 자연어로부터의 다른 유형의 policy 생성

### 1. 여러 줄 문장
여러 줄 문장이 제공되면 여러 policy가 생성되어 result['generatedPolicies']에 표시됩니다. 일관되게 나타나는 구분 기호(쉼표, 마침표, 세미콜론 등)가 개별 policy 문장을 구분하는 데 사용됩니다


In [ ]:
print("\n📝 Multi-line statement")

nl_input = """Allow all users to invoke the risk model tool when data governance approval is true. 
Block users from calling the application tool unless coverage amount is present"""

# 여러 줄 문장에서 여러 policy 생성
result = policy_client.generate_policy(
    policy_engine_id=config["policy_engine_id"],
    name=f"nl_policy_{int(time.time())}",
    resource={"arn": config["gateway"]["gateway_arn"]},
    content={"rawText": nl_input},
    fetch_assets=True,
)

# 생성된 모든 policy 출력
if result.get("status") == "GENERATED" and result.get("generatedPolicies"):
    for generated_policy in result["generatedPolicies"]:
        cedar_statement = (
            generated_policy.get("definition", {})
            .get("cedar", {})
            .get("statement", "N/A")
        )

        print("Generated Cedar Policy:")
        print("=" * 60)
        print(cedar_statement)
        print("=" * 60)

### 2. Principal 관련 Policy 문장
OAuth 액세스 토큰 형태로 idP 표현을 통해 전달되는 principal 범위를 기반으로 조건을 주장하는 policy를 생성할 수 있습니다. OAuth 토큰에 저장되는 속성을 사용자 정의할 수 있으므로, NL2Cedar 생성을 위해 정확한 태그를 제공하면 NL2Cedar가 올바른 Cedar policy를 생성하는 데 도움이 됩니다

In [ ]:
print("\n📝 Principal Scope statements")

nl_inputs = [
    'Allow principals with username "test-user" to invoke the risk model tool',
    str(
        'Forbid principals to access the approval tool unless they have the scope group:Controller <idp_claims>["scope"]</idp_claims>'
    ),
    str(
        'Block principals from using risk model tool and approval tool unless the principal has role "senior-adjuster"'
    ),
]

# 각 자연어 문장에 대해 policy 생성
for nl_input in nl_inputs:
    result = policy_client.generate_policy(
        policy_engine_id=config["policy_engine_id"],
        name=f"nl_policy_{int(time.time())}",
        resource={"arn": config["gateway"]["gateway_arn"]},
        content={"rawText": nl_input},
        fetch_assets=True,
    )

    if result.get("status") == "GENERATED" and result.get("generatedPolicies"):
        for generated_policy in result["generatedPolicies"]:
            cedar_statement = (
                generated_policy.get("definition", {})
                .get("cedar", {})
                .get("statement", "N/A")
            )
            print("=" * 60)
            print(f"Natural Language: {nl_input}")
            print("Generated Cedar Policy:")
            print("=" * 60)
            print(cedar_statement)
            print("=" * 60)

# 정리

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

with open(config_file, "r") as f:
    config = json.load(f)

# Policy Engine 먼저 정리
print("🧹 Cleaning up Policy Engine...")
policy_client = PolicyClient(region_name=config["region"])
policy_client.cleanup_policy_engine(config["policy_engine_id"])
print("✓ Policy Engine cleaned up\n")

# Gateway 정리
print("🧹 Cleaning up Gateway...")
gateway_client = GatewayClient(region_name=config["region"])
gateway_client.cleanup_gateway(
    config["gateway"]["gateway_id"], config["gateway"]["client_info"]
)
print("✅ Cleanup complete!")